In [107]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [108]:
annFile = '/home/coco/annotations/cocojson_val_1.json'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [109]:
result_path3 = "/home/mmdetection/work_dirs/ensemble_1_soft_nms.pkl"
with open(result_path3,"rb") as f:
    dt = pickle.load(f)

In [110]:
def bb_intersection_over_union(boxA, boxB):
   # determine the (x, y)-coordinates of the intersection rectangle
   # 画个图会很明显，x左、y上取大的，x右、y下取小的，刚好对应交集
   xA = max(boxA[0], boxB[0])
   yA = max(boxA[1], boxB[1])
   xB = min(boxA[2], boxB[2])
   yB = min(boxA[3], boxB[3])
   # compute the area of intersection rectangle
   # 计算交集部分面积
   interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
   # compute the area of both the prediction and ground-truth rectangles
   # 计算预测值和真实值的面积
   boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
   boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
   # compute the intersection over union by taking the intersection
   # area and dividing it by the sum of prediction + ground-truth
   # areas - the interesection area
   # 计算IoU，即 交/（A+B-交）
   iou = interArea / float(boxAArea + boxBArea - interArea)
   # return the intersection over union value
   return iou

In [111]:

score_thr = 0
iou_thr = 0

In [112]:
def compute_max_iou(gt,dt):
    # gt [1949.0, 0.0, 2941.0, 3005.0]
    # dt shape(n,4)
    tp = 0
    fp = 0
    for i in range(dt.shape[0]):
        iou = bb_intersection_over_union(gt,dt[i])
        if iou>iou_thr:
            tp=1
        else:
            fp+=1
    fn = 1-tp
    return tp,fp,fn

In [113]:
def compute_recall(imgId,anns):
    tp = 0
    fp = 0
    fn = 0
    tmp_img = []
    for i in anns:
        tmp = {}
        cid = i["category_id"]
        det_bboxes = dt[imgId][cid]
        x = det_bboxes[:,4] > score_thr
        det_bboxes = det_bboxes[x][:,:4]
        tp1,fp1,fn1 = compute_max_iou(i["bbox"],det_bboxes)
        tp +=tp1
        fp +=fp1
        fn +=fn1
    return tp,fp,fn

In [114]:
def compute_recall_cat(cat,imgId,anns):
    tp = 0
    fp = 0
    fn = 0
    tmp_img = []
    for i in anns:
        tmp = {}
        cid = i["category_id"]
        if cid !=cat:
            continue
        det_bboxes = dt[imgId][cid]
        x = det_bboxes[:,4] > score_thr
        det_bboxes = det_bboxes[x][:,:4]
        tp1,fp1,fn1 = compute_max_iou(i["bbox"],det_bboxes)
        tp +=tp1
        fp +=fp1
        fn +=fn1
    return tp,fp,fn

In [115]:
def compute_AR():
    tp = 0
    fp = 0
    fn = 0
    for i in range(1124):
        annIds = cocoGt.getAnnIds(imgIds=i)
        anns = cocoGt.loadAnns(annIds)
        tp1,fp1,fn1 = compute_recall(i,anns)
        tp +=tp1
        fp +=fp1
        fn +=fn1
    return tp/(tp+fp),tp/(tp+fn)

In [116]:
def compute_AR_cat(cat):
    tp = 0
    fp = 0
    fn = 0
    for i in range(1124):
        annIds = cocoGt.getAnnIds(imgIds=i)
        anns = cocoGt.loadAnns(annIds)
        tp1,fp1,fn1 = compute_recall_cat(cat,i,anns)
        tp +=tp1
        fp +=fp1
        fn +=fn1
    return tp/(tp+fp),tp/(tp+fn)

In [117]:
compute_AR()

(0.027276569459345232, 0.4869413813116657)

In [118]:
compute_AR_cat(0)

(0.014750198255352894, 0.41797752808988764)

In [119]:
compute_AR_cat(1)

(0.03440691280439906, 0.553030303030303)

In [120]:
compute_AR_cat(2)

(0.02336937565399372, 0.4110429447852761)

In [121]:
compute_AR_cat(3)

(0.04115733991252663, 0.5104311543810849)

# single

In [5]:
imgId = 234
cocoRoot= "/home/coco/train2017/"
imgInfo = cocoGt.loadImgs(imgId)[0]
imPath = os.path.join(cocoRoot, imgInfo['file_name'])   

In [42]:
annIds = cocoGt.getAnnIds(imgIds=imgInfo['id'])
print(f'图像{imgInfo["id"]}包含{len(annIds)}个ann对象，分别是:')
anns = cocoGt.loadAnns(annIds)
for i in anns:
    print(i["bbox"])

图像234包含1个ann对象，分别是:
[1949.0, 0.0, 2941.0, 3005.0]


In [49]:
score_thr = 0.3
iou_thr = 0.3
tmp_img = []
for i in anns:
    tmp = {}
    cid = i["category_id"]
    det_bboxes = dt[imgId][cid]
    x = det_bboxes[:,4] > score_thr
    det_bboxes = det_bboxes[x][:,:4]
    iou = compute_max_iou(i["bbox"],det_bboxes)
    tmp["category_id"] = cid
    tmp["iou"] = iou
    if iou>iou_thr:
        tmp["tp"] = True
    tmp_img.append(tmp)
print(tmp_img)

[{'category_id': 3, 'iou': 0.3268248177700389, 'tp': True}]


# recall = tp / (tp+fn)

In [50]:
tp = 0
for i in tmp_img:
    if i["tp"]:
        tp+=1
print("recall = ", tp/len(tmp_img))

recall =  1.0


In [63]:
import glob
ls = glob.glob("/home/coco/train2017/*")
import cv2
img = cv2.imread(ls[0])
img.shape

(3684, 4912, 3)

In [67]:
import pretrainedmodels

In [79]:
pretrainedmodels.model_names

['fbresnet152',
 'bninception',
 'resnext101_32x4d',
 'resnext101_64x4d',
 'inceptionv4',
 'inceptionresnetv2',
 'alexnet',
 'densenet121',
 'densenet169',
 'densenet201',
 'densenet161',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnet101',
 'resnet152',
 'inceptionv3',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19_bn',
 'vgg19',
 'nasnetamobile',
 'nasnetalarge',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn131',
 'dpn107',
 'xception',
 'senet154',
 'se_resnet50',
 'se_resnet101',
 'se_resnet152',
 'se_resnext50_32x4d',
 'se_resnext101_32x4d',
 'cafferesnet101',
 'pnasnet5large',
 'polynet']

In [122]:
backbone = pretrainedmodels.__dict__["resnet50"](num_classes=1000, pretrained='imagenet')
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [84]:
layer0 = backbone.layer0
layer1 = backbone.layer1
layer2 = backbone.layer2
layer3 = backbone.layer3

In [85]:
import torch
a = torch.rand((32,3,1200,800))
b = layer0(a)
b.shape

torch.Size([32, 64, 300, 200])

In [86]:
c = layer1(b)
c.shape

torch.Size([32, 256, 300, 200])

In [87]:
d = layer2(c)
d.shape

torch.Size([32, 512, 150, 100])

In [88]:
e = layer3(d)
e.shape

torch.Size([32, 1024, 75, 50])